<a href="https://colab.research.google.com/github/SangminAhn21/DL_Study/blob/main/Kaggle/Facial_Keypoint_Detection/FKD_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
%cd drive/MyDrive/Colab Notebooks/Facial_Keypoint_Detection

/content/drive/MyDrive/Colab Notebooks/Facial_Keypoint_Detection


In [4]:
pip install kaggle

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [6]:
!kaggle competitions download -c facial-keypoints-detection

test.zip: Skipping, found more recently modified local copy (use --force to force download)
SampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
training.zip: Skipping, found more recently modified local copy (use --force to force download)
IdLookupTable.csv: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip training.zip
!unzip test.zip

Archive:  training.zip
replace training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  test.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [12]:
import models, utils

In [8]:
import numpy as np
import pandas as pd



training_pd = pd.read_csv('training.csv')
test_pd = pd.read_csv('test.csv')

training_pd = training_pd.fillna(method='ffill')

training = training_pd.to_numpy()
test = test_pd.to_numpy()

train_image = training[:, -1]
train_key = training[:, :-1].astype('float64')
test_image = test[:, 1]

train_image = np.array([np.array([int(pixel) for pixel in image.split()]).\
                        reshape(96, 96) for image in train_image])
test_image = np.array([np.array([int(pixel) for pixel in image.split()]).\
                       reshape(96, 96) for image in test_image])

In [9]:
pip install ray

     |████████████████████████████████| 54.7 MB 34 kB/s 
     |████████████████████████████████| 118 kB 46.5 MB/s 


In [32]:
pip install -U tensorboardx

     |████████████████████████████████| 124 kB 5.5 MB/s 


In [35]:
from utils import FaceDataset, RMSELoss
from functools import partial
from models import CNN
import numpy as np
import torch
from torch.utils.data import DataLoader
import os
from filelock import FileLock
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


def cnn_train(config, data, checkpoint_dir=None, data_dir=None):
    if torch.cuda.is_available():
        DEVICE = torch.device('cuda')
    else:
        DEVICE = torch.device('cpu')
    print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

    model = CNN(config['l1'], config['l2']).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = config['lr'])
    criterion = RMSELoss()
    print(model)


    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)


    data_dir = os.path.abspath("./data")
    dataset = FaceDataset(data[0], data[1])

    lengths = [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)]
    train_data, val_data = torch.utils.data.random_split(dataset, lengths)

    train_loader = DataLoader(dataset=train_data,
                          batch_size=config['batch_size'],
                          shuffle=True,
                          num_workers=2)
    val_loader = DataLoader(dataset=val_data,
                        batch_size=config['batch_size'],
                        shuffle=True,
                        num_workers=2)

    # model.train()
    for Epoch in range(20):
        # running_loss = 0.0
        # epoch_steps = 0
        for batch_idx, (image, key) in enumerate(train_loader):
            image = image.to(DEVICE)
            key = key.to(DEVICE)
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, key)
            loss.backward()
            optimizer.step()

            # running_loss += loss.item()
            # epoch_steps += 1

            if batch_idx % 100 == 0:
                print("Train Epoch: {} [{}/{}({:.0f}%)]\tTrain Loss: {:.6f}".format(
                    Epoch, batch_idx * len(image),
                    len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                    loss.item()))
                # running_loss = 0.0

        # model.eval()
        val_loss = 0.0
        for image, key in val_loader:
            with torch.no_grad():
                image = image.to(DEVICE)
                key = key.to(DEVICE)
                output = model(image)
                val_loss += criterion(output, key).item()

        val_loss /= len(val_loader.dataset)
        val_loss *= config['batch_size']
        print('\n[EPOCH: {}], \tVal Loss: {:.4f}\n'.
        format(Epoch, val_loss))

        with tune.checkpoint_dir(Epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=val_loss)
    print("Finished Training")

In [36]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath("./data")  # 특정 경로에 대해 절대 경로 얻기
    config = {
        'l1': tune.sample_from(lambda _: 2**np.random.randint(3, 8)),
        'l2': tune.sample_from(lambda _: 2**np.random.randint(3, 8)),
        'lr': tune.loguniform(1e-3, 1e-1),
        'batch_size': tune.choice([8, 16, 32])
    }
    scheduler = ASHAScheduler(
        metric='loss',
        mode='min',
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "training_iteration"])
    
    result = tune.run(
        tune.with_parameters(partial(cnn_train, data_dir=data_dir), data=(train_image, train_key)),
        resources_per_trial={'cpu': 1, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))

    best_trained_model = CNN(best_trial.config["l1"], best_trial.config["l2"])
    if torch.cuda.is_available():
        DEVICE = torch.device('cuda')
    else:
        DEVICE = torch.device('cpu')
    best_trained_model.to(DEVICE)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    PATH = '/content/drive/MyDrive/Colab Notebooks/Facial_Keypoint_Detection/saved_model/'
    torch.save(best_trained_model, PATH + str(best_trial.config) + 'val_loss: ' + str(best_trial.last_result['loss']))

if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=20, gpus_per_trial=1)

== Status ==
Current time: 2021-11-19 07:26:07 (running for 00:00:00.21)
Memory usage on this node: 3.8/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/tune_with_parameters_2021-11-19_07-26-07
Number of trials: 10/10 (10 PENDING)
+----------------------------------+----------+-------+--------------+------+------+------------+
| Trial name                       | status   | loc   |   batch_size |   l1 |   l2 |         lr |
|----------------------------------+----------+-------+--------------+------+------+------------|
| tune_with_parameters_f6201_00000 | PENDING  |       |           16 |   64 |   32 | 0.0323551  |
| tune_with_parameters_f6201_00001 | PENDING  |       |           64 |    8 |   16 | 0.0317033  |
| tune_with_parameters_f6201_00002 

(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(raylet) /usr/local/lib/python3.7/dist-packages/redis/connection.py:72: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)


(scheduler +14m28s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(ImplicitFunc pid=1570) Using PyTorch version: 1.10.0+cu111  Device:  cuda
== Status ==
Current time: 2021-11-19 07:26:12 (running for 00:00:05.22)
Memory usage on this node: 4.9/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/tune_with_parameters_2021-11-19_07-26-07
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+----------------------------------+----------+-----------------+--------------+------+------+------------+
| Trial name                       | status   | loc             |   b

(pid=1570) /content/drive/My Drive/Colab Notebooks/Facial_Keypoint_Detection/utils.py:11: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
(pid=1570)   self.x_data = (torch.from_numpy(x)/255.).type('torch.FloatTensor')


(ImplicitFunc pid=1570) Train Epoch: 0 [0/5639(0%)]	Train Loss: 51.346024
(ImplicitFunc pid=1570) Train Epoch: 0 [1600/5639(28%)]	Train Loss: 5.985302
(ImplicitFunc pid=1570) Train Epoch: 0 [3200/5639(57%)]	Train Loss: 2.934462
(ImplicitFunc pid=1570) Train Epoch: 0 [4800/5639(85%)]	Train Loss: 5.582454
Result for tune_with_parameters_f6201_00000:
  date: 2021-11-19_07-26-17
  done: false
  experiment_id: ba0393095888428797db4ab915fd4ebd
  hostname: ecc602ac4891
  iterations_since_restore: 1
  loss: 3.265287729357997
  node_ip: 172.28.0.2
  pid: 1570
  should_checkpoint: true
  time_since_restore: 6.37751030921936
  time_this_iter_s: 6.37751030921936
  time_total_s: 6.37751030921936
  timestamp: 1637306777
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f6201_00000
  
(ImplicitFunc pid=1570) 
(ImplicitFunc pid=1570) [EPOCH: 0], 	Val Loss: 3.2653
(ImplicitFunc pid=1570) 
(ImplicitFunc pid=1570) Train Epoch: 1 [0/5639(0%)]	Train Loss: 3.125282
(ImplicitFunc pid=1570) Tra

...

(ImplicitFunc pid=3342) Using PyTorch version: 1.10.0+cu111  Device:  cuda
== Status ==
Current time: 2021-11-19 07:31:26 (running for 00:05:18.72)
Memory usage on this node: 5.0/12.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 16.000: -2.835878148315646 | Iter 8.000: -3.061555337567701 | Iter 4.000: -3.1204218276003575 | Iter 2.000: -3.376697772952682 | Iter 1.000: -3.265287729357997
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/tune_with_parameters_2021-11-19_07-26-07
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+----------------------------------+------------+-----------------+--------------+------+------+------------+---------+----------------------+
| Trial name                       | status     | loc             |   batch_size |   l1 |   l2 |         lr |    loss |   training_iteration |
|----------------------------------+------------+-----------------+---------

(pid=3342) /content/drive/My Drive/Colab Notebooks/Facial_Keypoint_Detection/utils.py:11: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
(pid=3342)   self.x_data = (torch.from_numpy(x)/255.).type('torch.FloatTensor')


(ImplicitFunc pid=3342) Train Epoch: 0 [0/5639(0%)]	Train Loss: 51.851734
(ImplicitFunc pid=3342) Train Epoch: 0 [1600/5639(28%)]	Train Loss: 3.470809
(ImplicitFunc pid=3342) Train Epoch: 0 [3200/5639(57%)]	Train Loss: 3.304157
(ImplicitFunc pid=3342) Train Epoch: 0 [4800/5639(85%)]	Train Loss: 2.886860
Result for tune_with_parameters_f6201_00009:
  date: 2021-11-19_07-31-30
  done: false
  experiment_id: 5de7d7968e874b1eb01832bdc7c8ac0c
  hostname: ecc602ac4891
  iterations_since_restore: 1
  loss: 2.8633132664024408
  node_ip: 172.28.0.2
  pid: 3342
  should_checkpoint: true
  time_since_restore: 6.38401985168457
  time_this_iter_s: 6.38401985168457
  time_total_s: 6.38401985168457
  timestamp: 1637307090
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f6201_00009
  
(ImplicitFunc pid=3342) 
(ImplicitFunc pid=3342) [EPOCH: 0], 	Val Loss: 2.8633
(ImplicitFunc pid=3342) 
(ImplicitFunc pid=3342) Train Epoch: 1 [0/5639(0%)]	Train Loss: 2.662274
(ImplicitFunc pid=3342) Tr

2021-11-19 07:32:34,478	INFO tune.py:630 -- Total run time: 387.04 seconds (386.87 seconds for the tuning loop).


Result for tune_with_parameters_f6201_00009:
  date: 2021-11-19_07-32-34
  done: true
  experiment_id: 5de7d7968e874b1eb01832bdc7c8ac0c
  hostname: ecc602ac4891
  iterations_since_restore: 20
  loss: 2.247820578230188
  node_ip: 172.28.0.2
  pid: 3342
  should_checkpoint: true
  time_since_restore: 70.5871012210846
  time_this_iter_s: 3.3550705909729004
  time_total_s: 70.5871012210846
  timestamp: 1637307154
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: f6201_00009
  
(ImplicitFunc pid=3342) 
(ImplicitFunc pid=3342) [EPOCH: 19], 	Val Loss: 2.2478
(ImplicitFunc pid=3342) 
== Status ==
Current time: 2021-11-19 07:32:34 (running for 00:06:26.91)
Memory usage on this node: 5.0/12.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 16.000: -2.5914811218883975 | Iter 8.000: -3.028511691262536 | Iter 4.000: -3.036302745088618 | Iter 2.000: -3.2002449783027593 | Iter 1.000: -3.231167747281122
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.36 GiB heap, 0.0/3.68 GiB obj

In [37]:
best_model = torch.load("/content/drive/MyDrive/Colab Notebooks/Facial_Keypoint_Detection/saved_model/{'l1': 128, 'l2': 16, 'lr': 0.005391064065783462, 'batch_size': 16}val_loss: 2.247820578230188")

In [13]:
import matplotlib.pyplot as plt

In [27]:
def predict(model, test_image, idx, plot=False):
    image_pred = torch.from_numpy(test_image[idx]).type(torch.FloatTensor).to(DEVICE)
    image_pred /= 255.
    with torch.no_grad():
        prediction = model(image_pred)
    image_pred = image_pred.cpu().numpy()*255.
    prediction = prediction.cpu().numpy().reshape(30)
    # answer = val_data[idx][1]

    if plot:
        fig, axis = plt.subplots()
        prediction_plot(image_pred, prediction, axis,\
                        'prediction plot for {}th image in val_data'.
                        format(idx))


def prediction_plot(image, keypoint, axis, title, answer=None):
    image = image.reshape(96, 96)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], marker='x', s=20)
    # axis.scatter(answer[0::2], answer[1::2], marker='x', color='r', s=20)
    plt.title(title)


In [15]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [121]:
predict(model=best_model, test_image=test_image, idx=174*, plot=True)

SyntaxError: ignored

In [43]:
def submit(model, test_image ):
    test_image = torch.from_numpy(test_image).type(torch.FloatTensor).to(DEVICE)
    test_image /= 255.
    with torch.no_grad():
        prediction = model(test_image)
    prediction = prediction.cpu().numpy()
    return prediction

In [76]:
pred = submit(best_model, test_image)

In [77]:
pred.shape

(1783, 30)

In [48]:
pd.read_csv('SampleSubmission.csv')

,RowId,Location
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27119,27120,0
27120,27121,0
27121,27122,0
27122,27123,0


In [80]:
IdLookupTable = pd.read_csv('IdLookupTable.csv')
IdLookupTable

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN
...,...,...,...,...
27119,27120,1783,right_eye_center_y,NaN
27120,27121,1783,nose_tip_x,NaN
27121,27122,1783,nose_tip_y,NaN
27122,27123,1783,mouth_center_bottom_lip_x,NaN


In [ ]:
RowID = list(IdLookupTable.RowId)
ImageID = list(IdLookupTable.ImageId)
FeatureHead = list(training_pd.columns.values)
FeatureIndex = [FeatureHead.index(feature) for feature in IdLookupTable.FeatureName]

In [ ]:
ImageID

In [133]:
location = []
for image_id, feature_id in zip(ImageID, FeatureIndex):
    location.append(pred[image_id-1][feature_id])
submission = pd.DataFrame({'RowID': RowID, 'Location': location})
submission.head()

,RowID,Location
0,1,67.517288
1,2,37.388115
2,3,29.673445
3,4,36.826153
4,5,60.113571


In [134]:
def adjust_max(row):
    if row.Location > 96.:
        row.Location = 96.
    return row

submission = submission.apply(adjust_max, axis='columns')

In [137]:
submission.RowID = submission.RowID.astype('int')

In [143]:
submission.head()

,RowID,Location
0,1,67.517288
1,2,37.388115
2,3,29.673445
3,4,36.826153
4,5,60.113571


In [144]:
submission.to_csv('submission_1.csv', index=None)

In [142]:
!kaggle competitions submit -c facial-keypoints-detection -f submission.csv -m "First Submission at Kaggle"

100% 628k/628k [00:01<00:00, 576kB/s] 
Successfully submitted to Facial Keypoints Detection